In [38]:
from sqlalchemy import create_engine
import pandas as pd

# Crear el motor de conexión a la base de datos Sakila
def connect_to_sakila(user, password, host, database):
    connection_string = f"mysql+pymysql://{user}:{password}@{host}/{database}"
    engine = create_engine(connection_string)  
    return engine

In [39]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT customer_id, COUNT(rental_id) AS rentals
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    GROUP BY customer_id;
    """
    return pd.read_sql(query, engine)


In [40]:
def rental_count_month(rentals_df, month, year):
    column_name = f"rentals_{month:02d}_{year}"
    grouped_df = rentals_df.groupby('customer_id').sum().reset_index()
    grouped_df.rename(columns={'rentals': column_name}, inplace=True)
    return grouped_df

In [41]:
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    rental_columns = [col for col in combined_df.columns if col.startswith('rentals')]
    combined_df['difference'] = combined_df[rental_columns[0]] - combined_df[rental_columns[1]]
    return combined_df

In [ ]:
# Conecto con la base de datos Sakila
engine = connect_to_sakila('root', 'password', 'localhost', 'sakila')

# Recupero datos para mayo y junio de 2005
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)

In [ ]:
may_counts = rental_count_month(may_rentals, 5, 2005)
june_counts = rental_count_month(june_rentals, 6, 2005)

In [ ]:
comparison = compare_rentals(may_counts, june_counts)